In [3]:
import pandas as pd

In [180]:
in_files = [
    "a_example.txt"
    , "b_read_on.txt"
    , "c_incunabula.txt"
    , "d_tough_choices.txt"
    , "e_so_many_books.txt"
    , "f_libraries_of_the_world.txt"
]

In [189]:
def to_int_list(line):
    return [b for b in map(int, line.strip().split(" "))]

def read_input(in_file):
    with open(in_file, 'r') as f:
        l = f.readline().strip()
        first_data = [d for d in map(int, l.split(" "))]
        B = first_data[0]
        L = first_data[1]
        D = first_data[2]
        print(f'Total Books: {B}\nLibraries: {L}\nDays: {D}')
        
        book_scores = [b for b in map(int, f.readline().strip().split(" "))]

#        print(book_scores)
        libraries = []
        for idx, library in enumerate(range(L)):
            lib_data_1 = to_int_list(f.readline())
            number_of_books = lib_data_1[0]
            time = lib_data_1[1]
            b_per_day = lib_data_1[2]
#            print(f'{library} -> books: {number_of_books}, Time: {time}, BxDay: {b_per_day}')
            lib_books = to_int_list(f.readline())
#            print(f'{library} -> Books: {lib_books}')
            libraries.append({"Library": idx
                              , "Number of books": number_of_books
                              , "Startup time": time
                              , "Books per day": b_per_day
                              , "Books": lib_books
                             })
        
        return ({
            "Total Books": B
            , "Total Libraries": L
            , "Total Days": D
            , "Books Scores": book_scores
            , "Libraries": libraries
        })


In [105]:
foo = read_input("a_example.txt")

Total Books: 6
Libraries: 2
Days: 7
[1, 2, 3, 6, 5, 4]
0 -> books: 5, Time: 2, BxDay: 2
0 -> Books: [0, 1, 2, 3, 4]
1 -> books: 4, Time: 3, BxDay: 1
1 -> Books: [0, 2, 3, 5]


In [198]:
books = pd.DataFrame(foo['Books Scores'])

In [107]:
sorted_books = books.sort_values(by=0, ascending=False)

In [111]:
sorted_books

,0
3,6
4,5
5,4
2,3
1,2
0,1


In [116]:
libraries = pd.DataFrame(foo['Libraries'])

In [119]:
libraries.sort_values('Startup time', inplace=True)

In [120]:
libraries

,Library,Number of books,Startup time,Books per day,Books
0,0,5,2,2,"[0, 1, 2, 3, 4]"
1,1,4,3,1,"[0, 2, 3, 5]"


In [123]:
signed_libraries = libraries.shape[0]

In [124]:
signed_libraries

2

In [171]:
for idx, library in libraries.iterrows():
    first_line = f"{library['Library']} {len(library['Books'])}"
    second_line = " ".join([str(b) for b in library['Books']])
    print(f'{first_line}\n{second_line}')

0 5
0 1 2 3 4
1 4
0 2 3 5


In [194]:
def write_out_file(out_file, solution):
    with open(out_file, 'w') as o:
        o.write(f"{solution.shape[0]}\n")
        for idx, library in solution.iterrows():
            first_line = f"{library['Library']} {len(library['Books'])}\n"
            second_line = " ".join([str(b) for b in library['Books']]) + "\n"
#            print(f'{first_line}\n{second_line}')
            o.write(first_line)
            o.write(second_line)
    

In [179]:
write_out_file("foo.out", libraries)

0 5

0 1 2 3 4

1 4

0 2 3 5



# Naive

In [197]:
for problem in in_files[-1:]:
    print(problem)
    data_in = read_input(problem)
    libraries = pd.DataFrame(data_in['Libraries'])
    libraries.sort_values('Startup time', inplace=True)
    write_out_file(problem.replace(".txt", ".out"), libraries)
    print("\n\n")

f_libraries_of_the_world.txt
Total Books: 100000
Libraries: 1000
Days: 700





In [184]:
"foo.txt".replace(".txt", ".out")

'foo.out'

# More naive

In [326]:
def find_solution(in_file):
    problem_data = read_input(in_file)
    books = pd.DataFrame(problem_data['Books Scores'])
    libraries = pd.DataFrame(problem_data['Libraries'])
    libraries.sort_values('Startup time', inplace=True)
    solution = libraries
    for idx , library in libraries.iterrows():
        library_books = pd.DataFrame(library['Books']).set_index(0)
        scan_books = books.index.intersection(library_books.index)
        if len(scan_books) == 0:
            solution.drop(index=[library['Library']], inplace=True)
        else:
            books.drop(index=scan_books, inplace=True)
            solution.at[library['Library'], 'Books'] = list(scan_books)
        
    return solution
    


In [327]:
foo = find_solution("a_example.txt")

Total Books: 6
Libraries: 2
Days: 7


In [328]:
foo.head()

,Library,Number of books,Startup time,Books per day,Books
0,0,5,2,2,"[0, 1, 2, 3, 4]"
1,1,4,3,1,[5]


In [ ]:
for problem in in_files:
    print(problem)
    data_in = read_input(problem)
    libraries = pd.DataFrame(data_in['Libraries'])
    solution = find_solution(problem)
    write_out_file(problem.replace(".txt", ".out"), solution)
    print("\n\n")

a_example.txt
Total Books: 6
Libraries: 2
Days: 7
Total Books: 6
Libraries: 2
Days: 7



b_read_on.txt
Total Books: 100000
Libraries: 100
Days: 1000
Total Books: 100000
Libraries: 100
Days: 1000



c_incunabula.txt
Total Books: 100000
Libraries: 10000
Days: 100000
Total Books: 100000
Libraries: 10000
Days: 100000
